# Note
* conviene **tralasciare eventi sotto 2+ magnitudine** perche **riduce il traffico sul server**
* la risposta come **testo semplifica il parsing dei dati ricevuti**
* conviene copiare i dati in un db diviso tra **dati storici (non cambiano) e dati recenti (ultimo giorno)** che puo cambiare molto spesso
* il **db va partizionato** cosi i dati storici sono automaticamente inseriti nelle tabelle corrette **migliorando l'efficenza delle query**

# Call examples

<h1 style='color:magenta;font-family: "Times New Roman", Times, serif;font-size: 40px;'>DA IMPLEMENTARE LA LETTURA DIRETTA DA XML INGV</h1>

In [14]:
import xmltodict
import pandas as pd

In [2]:
#open the file
fileptr = open("working_files/response_1688110777216.xml","r")

In [3]:
#read xml content from the file
xml_content= fileptr.read()
print("XML content is:")
print(xml_content)

XML content is:
<?xml version="1.0" encoding="US-ASCII" standalone="yes"?>
  <q:quakeml xmlns:q="http://quakeml.org/xmlns/quakeml/1.2" xmlns="http://quakeml.org/xmlns/bed/1.2" xmlns:ingv="http://webservices.ingv.it/fdsnws/event/1">
    <eventParameters publicID="smi:webservices.ingv.it/fdsnws/event/1/query">
      <event publicID="smi:webservices.ingv.it/fdsnws/event/1/query?eventId=33760891">
        <type>earthquake</type>
        <description>
          <type>region name</type>
          <text>Tirreno Meridionale (MARE)</text>
        </description>
        <preferredMagnitudeID>smi:webservices.ingv.it/fdsnws/event/1/query?magnitudeId=121550241</preferredMagnitudeID>
        <preferredOriginID>smi:webservices.ingv.it/fdsnws/event/1/query?originId=113196041</preferredOriginID>
        <creationInfo>
          <agencyID>INGV</agencyID>
          <author>hew10_mole#MOD_EQASSEMBLE</author>
          <creationTime>2022-12-29T15:11:26</creationTime>
          <ingv:id_locator>480117</ingv

In [4]:
#change xml format to ordered dict
my_ordered_dict=xmltodict.parse(xml_content)
print("Ordered Dictionary is:")
# print(my_ordered_dict)

Ordered Dictionary is:


In [5]:
# print(my_ordered_dict['q:quakeml']['eventParameters']['event'])

In [6]:
events = my_ordered_dict['q:quakeml']['eventParameters']['event']

In [7]:
my_ordered_dict['q:quakeml']['eventParameters']['event'][0].keys()

dict_keys(['@publicID', 'type', 'description', 'preferredMagnitudeID', 'preferredOriginID', 'creationInfo', 'origin', 'magnitude'])

In [8]:
for i in range(2):
    print(events[i]['@publicID'].split('?')[1].split('=')[1])

33760891
33759321


In [9]:
# for event in events:
#     print(event['description'])

In [10]:
print("Location is:")
print(my_ordered_dict['q:quakeml']['eventParameters']['event'][0]['description'])

Location is:
{'type': 'region name', 'text': 'Tirreno Meridionale (MARE)'}


In [11]:
dati_scelti = []
for event in events:
    dati_scelti.append([event['@publicID'].split('?')[1].split('=')[1],
                        event['description']['text'],
                        event['origin']['time']['value'],
                        event['origin']['latitude']['value'],
                        event['origin']['longitude']['value']])

In [12]:
dati_scelti[0]

['33760891',
 'Tirreno Meridionale (MARE)',
 '2022-12-29T15:11:00.120000',
 '39.3627',
 '15.2153']

In [15]:
df = pd.DataFrame(dati_scelti,columns=['event_id','place','date','lat','lon'])
df

,event_id,place,date,lat,lon
0,33760891,Tirreno Meridionale (MARE),2022-12-29T15:11:00.120000,39.3627,15.2153
1,33759321,Tirreno Meridionale (MARE),2022-12-29T12:38:53.300000,39.3663,15.2048
2,33554561,Isole Eolie (Messina),2022-12-04T07:12:45.370000,38.354,14.859
3,33545491,Bosnia and Herz. [Land],2022-12-02T20:05:06.870000,43.0918,18.1082
4,33508291,Bosnia and Herz. [Land],2022-11-28T07:49:50.077000,44.456,17.8333
5,33456501,Malta (MALTA),2022-11-22T12:40:33.230000,36.034,14.1937
6,33431491,Costa Marchigiana Pesarese (Pesaro-Urbino),2022-11-20T05:20:30.250000,43.9027,13.2642
7,33332891,Costa Marchigiana Anconetana (Ancona),2022-11-10T17:54:12.160000,43.9155,13.3372
8,33302041,Costa Marchigiana Pesarese (Pesaro-Urbino),2022-11-09T06:12:57.730000,43.9662,13.3015
9,33301931,Costa Marchigiana Anconetana (Ancona),2022-11-09T06:08:28.990000,43.9133,13.3447
